In [1]:
import pandas as pd
import numpy as np
import os
import sys
import re
import gc
import time
import warnings
warnings.filterwarnings("ignore")
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from collections import defaultdict 
from tqdm import tqdm

In [3]:
data_path = "../data/"
common_features_train_csv = os.path.join(data_path, "common_features_train.csv")
common_features_test_csv = os.path.join(data_path, "common_features_test.csv")

sample_skeleton_train_csv = os.path.join(data_path, "sample_skeleton_train.csv")
sample_skeleton_test_csv = os.path.join(data_path, "sample_skeleton_test.csv")

In [3]:
fea_cols = ['common_feature_index', 'feature_num2', 'feature_list2']
fea_train = pd.read_csv(common_features_train_csv, header=None, names=fea_cols)
#sample_cols = ['sample_id', 'click', 'conversion', 'common_feature_index', 'feature_num1', 'feature_list1']
#sample_train = pd.read_csv(sample_skeleton_train_csv, nrows=100000, header=None, names=sample_cols)

In [4]:
sample_cols = ['sample_id', 'click', 'conversion', 'common_feature_index', 'feature_num1', 'feature_list1']
sample_train = pd.read_csv(sample_skeleton_train_csv, header=None, names=sample_cols, nrows=1000000)

In [5]:
sample_train.shape

(42300135, 6)

In [4]:
fea_train.shape
fea_train.head(2)

(730600, 3)

common_feature_index  feature_num2  \
0     84dceed2e3a667f8           343   
1     0000350f0c2121e7           811   

                                                                                         feature_list2  
0  101313191.012534387741.012634387791.012734387821.012838648851.012938648881.015...  
1  127_1437162241.94591127_1435146270.69315127_1437728710.69315127_1435432831.60944127_...

In [5]:
%%time
def feature_list_split(x):
    feature_dict = defaultdict(list)
    for fea in x.split('\x01'):
        field = re.split('\x02', fea)
        #fea = re.split('\x03', field[1])
        #feature_dict[field[0]].append({'feature_id':fea[0], 'value':fea[1]})
        feature_dict[field[0]].append(field[1])
    return feature_dict
#fea_train['fea_dict2'] =  fea_train['feature_list2'].map(feature_list_split)

Wall time: 0 ns


In [6]:
sample_train['fea_dict1'] =  sample_train['feature_list1'].map(feature_list_split)

In [9]:
sample_train['fea_dict1'][0].keys()

dict_keys(['210', '216', '301', '205', '206', '207', '508'])

In [15]:
sample_train['205'] = sample_train['fea_dict1'].map(lambda x:x['205']) # 商品ID
sample_train['206'] = sample_train['fea_dict1'].map(lambda x:x['206']) # 商品所属类目ID
sample_train['207'] = sample_train['fea_dict1'].map(lambda x:x['207']) # 商品所属店铺ID

sample_train['210'] = sample_train['fea_dict1'].map(lambda x:x['210']) # 商品关联用户意图ID
sample_train['216'] = sample_train['fea_dict1'].map(lambda x:x['216']) # 商品的品牌ID
sample_train['508'] = sample_train['fea_dict1'].map(lambda x:x['508']) # 109_14和206域的组合特征
sample_train['509'] = sample_train['fea_dict1'].map(lambda x:x['509']) # 110_14和207域的组合特征
sample_train['702'] = sample_train['fea_dict1'].map(lambda x:x['702']) # 127_14和216域的组合特征
sample_train['853'] = sample_train['fea_dict1'].map(lambda x:x['853']) # 150_14和210域的组合特征
sample_train['301'] = sample_train['fea_dict1'].map(lambda x:x['301']) # 业务场景信息的一种分类表示



In [18]:
sample_train[['205','206','702']].head()

,205,206,702
0,[41862221.0],[83167991.0],[]
1,[41868551.0],[83166761.0],[]
2,[44790031.0],[83165901.0],[98677624.02535]
3,[49339151.0],[83161771.0],[]
4,[49456631.0],[83165891.0],[]


In [20]:
for i in ['205','206','207', '210','216','508','509', '702', '853', '301']:
    print('-'*100 + i + '-'*100)
    sample_train['301'].map(lambda x: len(x)).value_counts()
    

----------------------------------------------------------------------------------------------------205----------------------------------------------------------------------------------------------------


1    1000000
Name: 301, dtype: int64

----------------------------------------------------------------------------------------------------206----------------------------------------------------------------------------------------------------


1    1000000
Name: 301, dtype: int64

----------------------------------------------------------------------------------------------------207----------------------------------------------------------------------------------------------------


...

1    1000000
Name: 301, dtype: int64

----------------------------------------------------------------------------------------------------853----------------------------------------------------------------------------------------------------


1    1000000
Name: 301, dtype: int64

----------------------------------------------------------------------------------------------------301----------------------------------------------------------------------------------------------------


1    1000000
Name: 301, dtype: int64

In [16]:
sample_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 18 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   sample_id             1000000 non-null  int64 
 1   click                 1000000 non-null  int64 
 2   conversion            1000000 non-null  int64 
 3   common_feature_index  1000000 non-null  object
 4   feature_num1          1000000 non-null  int64 
 5   feature_list1         1000000 non-null  object
 6   fea_dict1             1000000 non-null  object
 7   keys                  1000000 non-null  object
 8   205                   1000000 non-null  object
 9   206                   1000000 non-null  object
 10  207                   1000000 non-null  object
 11  210                   1000000 non-null  object
 12  216                   1000000 non-null  object
 13  508                   1000000 non-null  object
 14  509                   1000000 non-null  object
 15 

In [13]:
fea_train['fea_dict2'][0].keys()

dict_keys(['101', '125', '126', '127', '128', '129', '150_14', '121', '127_14', '109_14', '110_14', '122', '124'])

In [15]:
fea_train['101'] = fea_train['fea_dict2'].map(lambda x: x['101']) # 用户ID

fea_train['121'] = fea_train['fea_dict2'].map(lambda x: x['121']) # 用户的一种分类ID
fea_train['122'] = fea_train['fea_dict2'].map(lambda x: x['122']) # 用户的一种分类ID
fea_train['124'] = fea_train['fea_dict2'].map(lambda x: x['124']) # 用户性别分类ID

fea_train['125'] = fea_train['fea_dict2'].map(lambda x: x['125']) # 用户年龄分类ID
fea_train['126'] = fea_train['fea_dict2'].map(lambda x: x['126']) # 用户消费水平分类I
fea_train['127'] = fea_train['fea_dict2'].map(lambda x: x['127']) # 用户消费水平分类II
fea_train['128'] = fea_train['fea_dict2'].map(lambda x: x['128']) # 用户是否就业
fea_train['129'] = fea_train['fea_dict2'].map(lambda x: x['129']) # 用户地理信息分类ID

fea_train['150_14'] = fea_train['fea_dict2'].map(lambda x: x['150_14']) # 用户意图ID以及用户在该意图上的历史行为累积数量*
fea_train['127_14'] = fea_train['fea_dict2'].map(lambda x: x['127_14']) # 商品店铺ID以及用户在该店铺上的历史行为累积数量*
fea_train['109_14'] = fea_train['fea_dict2'].map(lambda x: x['109_14']) # 商品类目ID以及用户在该类目上的历史行为累积数量*
fea_train['110_14'] = fea_train['fea_dict2'].map(lambda x: x['110_14']) # 商品店铺ID以及用户在该店铺上的历史行为累积数量*



,101,121
0,[313191.0],[34386871.0]
1,[3923261.0],[34387251.0]
2,[],[34386581.0]
3,[651871.0],[34386581.0]
4,[2411891.0],[34386851.0]
